In [103]:
%run ../collect.py --root_dir=/Users/yangjunjie/work/fftisdf-for-dmet/fftisdf-with-dmet-main/benchmark/klno/diamond --filename=out.log --output=../data/diamond-klno.json
%run ../collect.py --root_dir=/Users/yangjunjie/work/fftisdf-for-dmet/fftisdf-with-dmet-main/benchmark/k-sos-mp2/diamond --filename=out.log --output=../data/diamond-k-sos-mp2.json
%run ../collect.py --root_dir=/Users/yangjunjie/work/fftisdf-for-dmet/fftisdf-with-dmet-main/benchmark/ref-kpts/diamond --filename=out.log --output=../data/diamond-ref-kpts.json

In [104]:
import os, sys, json
import pandas as pd

def load_data(f):
    # Load the JSON data as a dictionary first
    with open(f, 'r') as file:
        data = json.load(file)
    
    # Convert the nested dictionary to a list of records
    records = []
    for path, params in data.items():
        # Add the path as a field and merge with params
        record = {'path': path, **params}
        records.append(record)
    
    # Create DataFrame from records
    return pd.DataFrame(records)

d = load_data('../data/diamond-klno.json')
data_diamond_klno = d[d['method'] == 'fftisdf-60-14']

d = load_data('../data/diamond-k-sos-mp2.json')
data_diamond_k_sos_mp2 = d[d['method'] == 'fftisdf-60-14']

In [127]:
# plot all the results
nkpt = data_diamond_klno['nkpt'].unique()
nkpt.sort()
nkpt = nkpt[0:-1]

for nk in nkpt:
    colors = ['r', 'g', 'b']
    label = ['e_corr_klno_corr_os', 'e_corr_klno_corr_mp2', 'e_corr_klno_corr_ccsd', 'e_corr_klno_corr_ccsd_t']
    cols = data_diamond_klno[data_diamond_klno['nkpt'] == nk][label]
    x0 = data_diamond_k_sos_mp2[data_diamond_k_sos_mp2['nkpt'] == nk]['ene_os_lt30'].item()

    # print(f"nkpt: {nk}")
    cols = cols[cols != 0]
    # delete NaN
    cols = cols.dropna()
    # sort by e_corr_klno_corr_os
    cols = cols.sort_values('e_corr_klno_corr_os')


    import matplotlib.pyplot as plt
    # fig, ax = plt.subplots()

    ene_krhf = data_diamond_klno[data_diamond_klno['nkpt'] == nk]['ene_krhf'].values[0]
    out = "nk = %4d, ene_krhf = %12.8f, ene_sos_mp2 = %12.8f" % (nk, ene_krhf, ene_krhf + x0 * 1.3)
    for il, l in enumerate(label[1:]):
        x = cols['e_corr_klno_corr_os']
        y = cols[l]
        # delete repeated values
        x = x.drop_duplicates()
        y = y.drop_duplicates()
        # print(x)
        # print(y)
        
        import numpy, scipy
        from scipy.optimize import curve_fit

        r = numpy.polyfit(x[:4], y[:4], 1)
        p = numpy.poly1d(r)
        y0 = p(x0)

        # ax.scatter(x, y, c=colors[il], label=l)
        # xmin = min(x.min(), x0)
        # xmax = max(x.max(), x0)
        # x = numpy.linspace(xmin, xmax, 100)
        # y = p(x)
        # ax.plot(x, y, color=colors[il], linestyle='--', marker='')
        # ax.scatter([x0], [y0], c=colors[il], label=l, marker='x')

        # ax.legend()
        
        name = "ene_" + l.split('e_corr_klno_corr_')[-1]
        out += f", {name} = {y0 + ene_krhf:12.8f}"

    print(out)



nk =    2, ene_krhf = -10.57682410, ene_sos_mp2 = -10.79326588, ene_mp2 = -10.79447736, ene_ccsd = -10.82233978, ene_ccsd_t = -10.83176806
nk =    4, ene_krhf = -10.82926354, ene_sos_mp2 = -11.05036103, ene_mp2 = -11.05831548, ene_ccsd = -11.07575006, ene_ccsd_t = -11.08617015
nk =    8, ene_krhf = -10.95722433, ene_sos_mp2 = -11.17958906, ene_mp2 = -11.19198167, ene_ccsd = -11.20465634, ene_ccsd_t = -11.21554575
nk =   12, ene_krhf = -10.99956058, ene_sos_mp2 = -11.22787028, ene_mp2 = -11.24397339, ene_ccsd = -11.25539156, ene_ccsd_t = -11.26606734
nk =   18, ene_krhf = -11.02151491, ene_sos_mp2 = -11.25340190, ene_mp2 = -11.27047886, ene_ccsd = -11.28290776, ene_ccsd_t = -11.29260498
nk =   27, ene_krhf = -11.02227872, ene_sos_mp2 = -11.25604321, ene_mp2 = -11.27366813, ene_ccsd = -11.28550495, ene_ccsd_t = -11.29517368
nk =   36, ene_krhf = -11.02794901, ene_sos_mp2 = -11.26467121, ene_mp2 = -11.28405596, ene_ccsd = -11.29421058, ene_ccsd_t = -11.30422366
nk =   48, ene_krhf = -11.0